In [94]:
import sys, os
import numpy as np
import matplotlib.pyplot as plt
from dslab import adguide #source: https://github.com/adophobr/PhotonicIntegratedCircuits/blob/main/jupyter/utils/dslab.py
import warnings
sys.path.append('C:\\Program Files\\Lumerical\\v221\\api\\python')
sys.path.append('C:\\Program Files\\Lumerical\\v221\\api\\python\\lumapi.py')
from scipy.constants import c

In [95]:
warnings.filterwarnings("ignore", category=SyntaxWarning)
import lumapi
fdtdApi = lumapi.FDTD()

Units

In [96]:
um = 1e-6
nm = 1e-9

Materials

In [97]:
materialClad = "SiO2 (Glass) - Dispersive & Lossless"
materialSi = "Si (Silicon) - Dispersive & Lossless"

# Add material
matname = materialSi

fdtdApi.addmaterial("Lorentz")
fdtdApi.setmaterial('New material 1', "Name", matname)
fdtdApi.setmaterial(matname, "Permittivity", 7.9837492)
fdtdApi.setmaterial(matname, "Lorentz Linewidth", 1e8)
fdtdApi.setmaterial(matname, "Lorentz Resonance", 3.93282466e+15)
fdtdApi.setmaterial(matname, "Lorentz Permittivity", 3.68799143)
fdtdApi.setmaterial(matname, "color", np.array([0.85, 0, 0, 1])) # red

In [98]:
matname = materialClad
fdtdApi.addmaterial("Lorentz")
fdtdApi.setmaterial('New material 1', "Name", matname)
fdtdApi.setmaterial(matname, "Permittivity", 2.119881)
fdtdApi.setmaterial(matname, "Lorentz Linewidth", 1e10)
fdtdApi.setmaterial(matname, "Lorentz Resonance", 3.309238e+13)
fdtdApi.setmaterial(matname, "Lorentz Permittivity", 49.43721)
fdtdApi.setmaterial(matname, "color", np.array([0.5, 0.5, 0.5, 1])) # grey

Dimensions

In [99]:
Wm = 7*um
L  = 42.5*um #obtained via EME and varFDTD simulations
#tapers
taper_large_tip = 1.5 * um
taper_length = 15 * um
#slabs 
width = 450 * nm
height = 220 * nm
length = 5 * um
cband = True
# C band simulation: 1520 ~ 1575 nm
# L band simulation: 1555 ~ 1635 nm
if cband:
    wl_start = 1520 * nm
    wl_stop = 1575 * nm
else:
    wl_start = 1555 * nm
    wl_stop = 1635 * nm


In [100]:
fdtdApi.switchtolayout()
fdtdApi.deleteall()
fdtdApi.cleardcard

<bound method Lumerical.__init__.<locals>.<lambda>.<locals>.<lambda> of <lumapi.FDTD object at 0x000001F76E84DBB0>>

In [101]:
fdtdApi.addstructuregroup()
dev_layer = 'GDS_LAYER_35:4'
fdtdApi.set('name', dev_layer)
filename = 'FDTD_mmi_2x2_TE.dat'

Box

In [102]:
fdtdApi.addrect()
fdtdApi.addtogroup(dev_layer)
fdtdApi.set('name','box')
fdtdApi.set('x',0)
fdtdApi.set('x span', L)
fdtdApi.set('y',0)
fdtdApi.set('y span', Wm)
fdtdApi.set('z',height/2)
fdtdApi.set('z span', height)
fdtdApi.set('material', materialSi)

Tapers

In [103]:
for i in range(4):
    fdtdApi.addobject('linear_taper')
    fdtdApi.set('name', f'taper port {str(i+1)}')
    fdtdApi.set('thickness', height)
    fdtdApi.set('angle_side', 90)
    fdtdApi.set('width_l', width)
    fdtdApi.set('width_r', taper_large_tip)
    fdtdApi.set('len', taper_length)
    fdtdApi.set('material', materialSi)
    fdtdApi.set('x', -(fdtdApi.getnamed(f'{dev_layer}::box', 'x max') + taper_length/2))
    fdtdApi.set('y', Wm/4)
    fdtdApi.set('z', height/2)

In [104]:
#adjusting port 2
fdtdApi.select(f'{dev_layer}::taper port 2')
fdtdApi.set('width_l', taper_large_tip)
fdtdApi.set('width_r', width)
fdtdApi.set('x', fdtdApi.getnamed(f'{dev_layer}::box', 'x max') + taper_length/2)
fdtdApi.set('y', Wm/4)

##adjusting port 3
fdtdApi.select(f'{dev_layer}::taper port 3')
fdtdApi.set('width_l', taper_large_tip)
fdtdApi.set('width_r', width)
fdtdApi.set('x', fdtdApi.getnamed(f'{dev_layer}::box', 'x max') + taper_length/2)
fdtdApi.set('y', -Wm/4)
#adjusting port 4

fdtdApi.select(f'{dev_layer}::taper port 4')
fdtdApi.set('y', -Wm/4)

Waveguides

In [105]:
for i in range(4):
    fdtdApi.addrect()
    fdtdApi.addtogroup(dev_layer)
    fdtdApi.set('name', f'port {str(i+1)}')
    fdtdApi.set('x', -(fdtdApi.getnamed(f'{dev_layer}::box', 'x max') + taper_length + length/2))
    fdtdApi.set('x span', length)
    fdtdApi.set('y', Wm/4)
    fdtdApi.set('y span', width)
    fdtdApi.set('z', height/2)
    fdtdApi.set('z span', height)
    fdtdApi.set('material', materialSi)

In [106]:
#adjusting
fdtdApi.select(f'{dev_layer}::port 2')
fdtdApi.set('x', (fdtdApi.getnamed(f'{dev_layer}::box', 'x max') + taper_length + length/2))

fdtdApi.select(f'{dev_layer}::port 3')
fdtdApi.set('x', (fdtdApi.getnamed(f'{dev_layer}::box', 'x max') + taper_length + length/2))
fdtdApi.set('y', -Wm/4)

fdtdApi.select(f'{dev_layer}::port 4')
fdtdApi.set('y',-Wm/4)

Adding solver

In [125]:
#simulation time
simulation_time = (L + 2*taper_length * 2*length)*7/c+200e-15
#position
x_FDTD = 0
y_FDTD = 0
z_FDTD = 0
#span
x_span_FDTD = (L + 2*taper_length + length) + 1*um
y_span_FDTD = Wm + 0.25*Wm
z_span_FDTD = height + 0.25*height
#material
background_material = materialClad
nmbFrqPts = 20
mesh_accuracy = 1

In [108]:
#ports parameters
y_port_off = 0.5*um

x_span_port = 4*um
z_span_port = 4*um
#ports positions
x_ports_positions = np.array([-(L/2 + taper_length + length/2),(L/2 + taper_length + length/2),(L/2 + taper_length + length/2),-(L/2 + taper_length + length/2)])
y_ports_positions = np.array([Wm/4, Wm/4, -Wm/4, -Wm/4])
mode_selection = 'fundamental TE mode'
injection_axis = 'x-axis'

Solver

In [109]:
fdtdApi.addfdtd()
fdtdApi.set('x', x_FDTD)
fdtdApi.set('x span',x_span_FDTD)
fdtdApi.set('y', y_FDTD)
fdtdApi.set('y span',y_span_FDTD)
fdtdApi.set('z', z_FDTD)
fdtdApi.set('z span',z_span_FDTD)
fdtdApi.set('background material', background_material)
fdtdApi.set('mesh accuracy', mesh_accuracy)
#boundary conditions
fdtdApi.set('x min bc', 'PML')
fdtdApi.set('x max bc', 'PML')
fdtdApi.set('y min bc', 'PML')
fdtdApi.set('y max bc', 'PML')
fdtdApi.set('z min bc', 'Symmetric')
fdtdApi.set('z max bc', 'PML')

Ports

In [112]:
for i in range(4):
    fdtdApi.addport()
    fdtdApi.set('name', f'port {str(i+1)}')
    fdtdApi.set('injection axis', injection_axis)
    fdtdApi.set('mode selection', mode_selection)
    fdtdApi.set('x', x_ports_positions[i])
    fdtdApi.set('y', y_ports_positions[i])
    fdtdApi.set('z', 0)
    fdtdApi.set('x span', x_span_port)
    fdtdApi.set('y span', 2*um)
    fdtdApi.set('z span', z_span_port)

In [117]:
fdtdApi.addprofile()
fdtdApi.set('override global monitor settings', True)
fdtdApi.set('frequency points', nmbFrqPts)
fdtdApi.set('x', x_FDTD)
fdtdApi.set('x span', x_span_FDTD)
fdtdApi.set('y', y_FDTD)
fdtdApi.set('y span', y_span_FDTD)
fdtdApi.set('z', 0)


In [121]:
fdtdApi.setnamed('FDTD','global source wavelength start', wl_start)
fdtdApi.setnamed('FDTD','global source wavelength stop', wl_stop)
fdtdApi.setnamed('FDTD::ports','monitor frequency points',nmbFrqPts)

In [122]:
fdtdApi.save(filename)

In [131]:
fdtdApi.run(3)

LumApiError: 'run can have at most 1 argument when first argument is a string'